In [ ]:
  !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr xgboost -q
  !pip install rouge
  !pip install --upgrade rouge-score

  from transformers import BartForConditionalGeneration, T5ForConditionalGeneration
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  import pandas as pd
  import numpy as np
  import torch
  from sklearn.metrics import mean_squared_error
  from rouge_score import rouge_scorer
  from transformers import PegasusForConditionalGeneration, AutoTokenizer
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  from rouge import Rouge
  from google.colab import drive
  from datasets import load_dataset
  import nltk
  import xgboost as xgb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Initialize the pegasus tokenizer and model
pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

# Initialize the pegasus tokenizer and model
T5_tokenizer = AutoTokenizer.from_pretrained("t5-large")

In [ ]:

df_pegasus_train=pd.read_csv('/content/drive/MyDrive/GP data/Pegasus_Summaries_Train(0-11490).csv')
df_t5_train=pd.read_csv('/content/drive/MyDrive/GP data/T5_TRAIN_SUMMARIES 0-11490.csv')
df_pegasus_test=pd.read_csv('/content/drive/MyDrive/GP data/Pegasus_Test_Summaries_0-11491.csv')
df_t5_test=pd.read_csv('/content/drive/MyDrive/GP data/T5_SUMMARIES_[0-11490].csv', encoding='ISO-8859-1')

In [ ]:
import nltk
import string

# download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# define function to clean text
def clean_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation+ '.'))
    # convert to lowercase
    text = text.lower()
    # tokenize text into words
    words = nltk.word_tokenize(text)
    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    words = [word for word in words if word not in stopwords]
    # join words back into a string
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# apply cleaning function to summary column
df_pegasus_train['summary'] = df_pegasus_train['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_pegasus_train['highlights'] = df_pegasus_train['highlights'].apply(clean_text)

In [ ]:
# apply cleaning function to summary column
df_pegasus_test['summary'] = df_pegasus_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_pegasus_test['highlights'] = df_pegasus_test['highlights'].apply(clean_text)

In [ ]:
# apply cleaning function to summary column
df_t5_test['summary'] = df_t5_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_t5_test['highlights'] = df_t5_test['highlights'].apply(clean_text)

In [ ]:
x_t5_train = []
for i, summary in df_t5_train['summary'].items():
    if isinstance(summary, str):
        encoded_summary = T5_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
        x_t5_train.append((i, encoded_summary.numpy().squeeze()))
    else:
        print(f"Summary at index {i} is not a string: {summary}")

In [ ]:
x_pegasus_train = []
for i, summary in df_pegasus_train['summary'].items():
    encoded_summary=pegasus_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_pegasus_train.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
y_train = []
for i, summary in df_pegasus_train['highlights'].items():
    encoded_summary = pegasus_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_train.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
x_t5_test = []
for i, summary in df_t5_test['summary'].items():
    encoded_summary =  pegasus_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_t5_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
x_pegasus_test = []
for i, summary in df_pegasus_test['summary'].items():
    encoded_summary = pegasus_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_pegasus_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
y_test = []
for i, summary in df_pegasus_test['highlights'].items():
    encoded_summary = pegasus_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
# Convert the input data to numpy arrays
x_t5_train = np.array(x_t5_train)
x_t5_test = np.array(x_t5_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_pegasus_train = np.array(x_pegasus_train)
x_pegasus_test = np.array(x_pegasus_test)

In [ ]:
x_t5_train_indices = np.array([t[0] for t in x_t5_train])
x_t5_train_summaries = np.array([t[1] for t in x_t5_train])
y_train_indices = np.array([t[0] for t in y_train])
y_train_summaries = np.array([t[1] for t in y_train])
x_pegasus_train_indices = np.array([t[0] for t in x_pegasus_train])
x_pegasus_train_summaries = np.array([t[1] for t in x_pegasus_train])

In [ ]:
x_t5_test_indices = np.array([t[0] for t in x_t5_test])
x_t5_test_summaries = np.array([t[1] for t in x_t5_test])
y_test_indices = np.array([t[0] for t in y_test])
y_test_summaries = np.array([t[1] for t in y_test])
x_pegasus_test_indices = np.array([t[0] for t in x_pegasus_test])
x_pegasus_test_summaries = np.array([t[1] for t in x_pegasus_test])

In [ ]:
# Stack the BART and Pegasus summaries
predictions_train = np.column_stack((x_pegasus_train_summaries[0:11490], x_t5_train_summaries))
predictions_test = np.column_stack((x_pegasus_test_summaries, x_t5_test_summaries[0:11489]))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Train a RandomForestRegressor on BART summaries
rf_base = RandomForestRegressor(n_estimators=1)
rf_base.fit(predictions_train, y_train_summaries[0:11490])

RandomForestRegressor(n_estimators=1)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# Train a KNeighborsRegressor on Pegasus summaries
mlp_base = MLPRegressor()
mlp_base.fit(predictions_train, y_train_summaries[0:11490])

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor()

In [ ]:
# Step 5: Create a new DataFrame to store the predictions
predictions = pd.DataFrame({"rf_preds": rf_base.predict(predictions_test[0:5000]).ravel(), "mlp_preds": mlp_base.predict(predictions_test[0:5000]).ravel()})

In [ ]:
y_test_flat = y_test_summaries[0:5000].ravel()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# Initialize base estimators
estimators = [
    ('rf',  RandomForestRegressor()),
    ('mlp', MLPRegressor()),
]


# Initialize meta-model with base estimators
meta_model =GradientBoostingRegressor(n_estimators=100)
meta_model.fit(predictions, y_test_flat)

GradientBoostingRegressor()

In [ ]:
test_rf_preds=rf_base.predict(predictions_test[5000:]).ravel()
test_mlp_preds=mlp_base.predict(predictions_test[5000:]).ravel()

In [ ]:
# Step 5: Create a new DataFrame to store the predictions
test_predictions = pd.DataFrame({"rf_preds": test_rf_preds, "mlp_preds":test_mlp_preds})

In [ ]:
ensemble_preds = meta_model.predict(test_predictions)

In [ ]:
ensemble_preds = ensemble_preds.reshape((6489,-1))

In [ ]:
# set min and max values for token IDs based on tokenizer vocab size
min_value = 0
max_value = pegasus_tokenizer.vocab_size - 1

# decode summaries from predicted token IDs
ensemble_pred_text = []
for i in range(len(ensemble_preds)):
    summary_ids = ensemble_preds[i]
    # clip token IDs to valid range
    summary_ids = np.clip(summary_ids, min_value, max_value)
    summary_ids = summary_ids.astype(int)
    summary = pegasus_tokenizer.decode(summary_ids, skip_special_tokens=True, max_length=100)
    ensemble_pred_text.append(summary)

In [ ]:
y_test_text = []
for i in range(len(y_test)):
    encoded_summary = y_test[i][1]
    decoded_summary = pegasus_tokenizer.decode(encoded_summary.squeeze(), skip_special_tokens=True,max_length=100)
    y_test_text.append( decoded_summary)

In [ ]:
rouge = Rouge()
scores = []
for i in range(len(ensemble_pred_text)):
    reference = str(y_test_text[i])
    hypothesis = str(ensemble_pred_text[i])
    if not reference:
        score = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
    else:
        score = rouge.get_scores(hypothesis, reference)
    scores.append(score)

In [ ]:
# Get the three ROUGE scores for each summary and calculate the average
rouge_1_scores = [score[0]["rouge-1"]["f"] for score in scores]
rouge_l_scores = [score[0]["rouge-l"]["f"] for score in scores]
average_rouge_1 = sum(rouge_1_scores) / (len(scores))
average_rouge_L = sum(rouge_l_scores) / (len(scores))

In [ ]:
# Save the ROUGE scores to a CSV file
rouge_scores_df = pd.DataFrame({
    "rouge-1": rouge_1_scores,
    "rouge-l": rouge_l_scores,
    "Average-1": average_rouge_1,
    "Average-L": average_rouge_L,
})
rouge_scores_df.to_csv("T5&Peg_Gradient_Rouge_Scores.csv", index=False)